In [13]:
# ============================================================================
# CELL 1: Imports and Setup
# ============================================================================
import pandas as pd
import numpy as np
import sqlite3
from pathlib import Path
import json

print("✅ All imports successful")

✅ All imports successful


In [14]:
# ============================================================================
# CELL 2: Load Ames Housing Data
# ============================================================================
data_path = Path('/Users/karthika/housing_app_fall25/data')
df = pd.read_csv(data_path / 'housing.csv')

print(f"📊 Original shape: {df.shape}")
print(f"📋 Columns: {len(df.columns)}")
print("\n🔍 First few rows:")
print(df.head())

📊 Original shape: (2930, 82)
📋 Columns: 82

🔍 First few rows:
   Order        PID  MS SubClass MS Zoning  Lot Frontage  Lot Area Street  \
0      1  526301100           20        RL         141.0     31770   Pave   
1      2  526350040           20        RH          80.0     11622   Pave   
2      3  526351010           20        RL          81.0     14267   Pave   
3      4  526353030           20        RL          93.0     11160   Pave   
4      5  527105010           60        RL          74.0     13830   Pave   

  Alley Lot Shape Land Contour  ... Pool Area Pool QC  Fence Misc Feature  \
0   NaN       IR1          Lvl  ...         0     NaN    NaN          NaN   
1   NaN       Reg          Lvl  ...         0     NaN  MnPrv          NaN   
2   NaN       IR1          Lvl  ...         0     NaN    NaN         Gar2   
3   NaN       Reg          Lvl  ...         0     NaN    NaN          NaN   
4   NaN       IR1          Lvl  ...         0     NaN  MnPrv          NaN   

  Misc Val M

In [15]:
# ============================================================================
# CELL 3: Create Classification Target
# ============================================================================
print("\n" + "="*70)
print("CREATING PRICE CATEGORIES")
print("="*70)

# Check if PriceCategory already exists
if 'PriceCategory' not in df.columns:
    # Calculate quartiles
    q1 = df['SalePrice'].quantile(0.25)
    q2 = df['SalePrice'].quantile(0.50)
    q3 = df['SalePrice'].quantile(0.75)
    
    print(f"Price Quartiles:")
    print(f"  Q1 (25%): ${q1:,.0f}")
    print(f"  Q2 (50%): ${q2:,.0f}")
    print(f"  Q3 (75%): ${q3:,.0f}")
    
    # Create categorical target
    def create_price_category(price):
        if price <= q1:
            return 0  # Low
        elif price <= q2:
            return 1  # Medium
        elif price <= q3:
            return 2  # High
        else:
            return 3  # Very High
    
    df['PriceCategory'] = df['SalePrice'].apply(create_price_category)
    print("\n✅ PriceCategory created")
else:
    print("✅ PriceCategory already exists in dataset")
    q1 = df[df['PriceCategory'] == 0]['SalePrice'].max()
    q2 = df[df['PriceCategory'] == 1]['SalePrice'].max()
    q3 = df[df['PriceCategory'] == 2]['SalePrice'].max()

# Verify distribution
print(f"\n📊 Class Distribution:")
print(df['PriceCategory'].value_counts().sort_index())



CREATING PRICE CATEGORIES
Price Quartiles:
  Q1 (25%): $129,500
  Q2 (50%): $160,000
  Q3 (75%): $213,500

✅ PriceCategory created

📊 Class Distribution:
PriceCategory
0    739
1    728
2    732
3    731
Name: count, dtype: int64


In [16]:
# ============================================================================
# CELL 4: Feature Selection and Cleaning
# ============================================================================
print("\n" + "="*70)
print("FEATURE SELECTION AND CLEANING")
print("="*70)

# Exclude ID columns and SalePrice (we have PriceCategory now)
columns_to_exclude = ['Order', 'PID', 'SalePrice'] if 'Order' in df.columns else ['Id', 'SalePrice']

# Get all available feature columns
available_features = [col for col in df.columns if col not in columns_to_exclude]

print(f"Total features (including target): {len(available_features)}")

# Create clean dataframe
df_clean = df[available_features].copy()

print(f"Cleaned dataset shape: {df_clean.shape}")



FEATURE SELECTION AND CLEANING
Total features (including target): 80
Cleaned dataset shape: (2930, 80)


In [17]:
# ============================================================================
# CELL 5: Handle Missing Values
# ============================================================================
print("\n" + "="*70)
print("HANDLING MISSING VALUES")
print("="*70)
print(f"Missing values before: {df_clean.isnull().sum().sum()}")

# For numeric columns: fill with median
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
numeric_cols = [col for col in numeric_cols if col != 'PriceCategory']

print(f"\n📊 Numeric columns: {len(numeric_cols)}")
for col in numeric_cols:
    if df_clean[col].isnull().any():
        median_val = df_clean[col].median()
        df_clean[col].fillna(median_val, inplace=True)

# For categorical columns: fill with 'Missing'
categorical_cols = df_clean.select_dtypes(include=['object']).columns.tolist()

print(f"📊 Categorical columns: {len(categorical_cols)}")
for col in categorical_cols:
    if df_clean[col].isnull().any():
        df_clean[col].fillna('Missing', inplace=True)

print(f"\n✅ Missing values after: {df_clean.isnull().sum().sum()}")

print("\n" + "="*70)
print("DATA SUMMARY")
print("="*70)
print(f"Total samples: {len(df_clean)}")
print(f"Total features: {len(df_clean.columns) - 1}")
print(f"Numeric features: {len(numeric_cols)}")
print(f"Categorical features: {len(categorical_cols)}")
print(f"Target classes: {df_clean['PriceCategory'].nunique()}")
print("="*70)




HANDLING MISSING VALUES
Missing values before: 15749

📊 Numeric columns: 36
📊 Categorical columns: 43

✅ Missing values after: 0

DATA SUMMARY
Total samples: 2930
Total features: 79
Numeric features: 36
Categorical features: 43
Target classes: 4


/var/folders/gr/p1sqxmsn6gb9tvm76vlf43_80000gn/T/ipykernel_87228/2916235485.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean[col].fillna(median_val, inplace=True)
/var/folders/gr/p1sqxmsn6gb9tvm76vlf43_80000gn/T/ipykernel_87228/2916235485.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

In [18]:
# ============================================================================
# CELL 6: Create SQLite Database
# ============================================================================
print("\n" + "="*70)
print("CREATING DATABASE")
print("="*70)

db_path = Path('/Users/karthika/housing_app_fall25/db/housing_classification.db')
db_path.parent.mkdir(exist_ok=True, parents=True)

# Remove old database if exists
if db_path.exists():
    db_path.unlink()
    print("🗑️  Removed old database")

# Create connection and table
conn = sqlite3.connect(db_path)
df_clean.to_sql('housing_data', conn, if_exists='replace', index=False)

print(f"✅ Database created: {db_path}")
print(f"   Total records: {len(df_clean)}")
print(f"   Total columns: {len(df_clean.columns)}")

# Verify database
test_query = pd.read_sql("SELECT * FROM housing_data LIMIT 5", conn)
print("\n📋 First 5 records from database:")
print(test_query.head())

# Check target distribution
target_dist = pd.read_sql(
    "SELECT PriceCategory, COUNT(*) as count FROM housing_data GROUP BY PriceCategory ORDER BY PriceCategory",
    conn
)
print("\n📊 Target distribution in database:")
for _, row in target_dist.iterrows():
    print(f"   Class {row['PriceCategory']}: {row['count']} samples")

conn.close()
print("\n✅ Database creation completed!")





CREATING DATABASE
🗑️  Removed old database
✅ Database created: /Users/karthika/housing_app_fall25/db/housing_classification.db
   Total records: 2930
   Total columns: 80

📋 First 5 records from database:
   MS SubClass MS Zoning  Lot Frontage  Lot Area Street    Alley Lot Shape  \
0           20        RL         141.0     31770   Pave  Missing       IR1   
1           20        RH          80.0     11622   Pave  Missing       Reg   
2           20        RL          81.0     14267   Pave  Missing       IR1   
3           20        RL          93.0     11160   Pave  Missing       Reg   
4           60        RL          74.0     13830   Pave  Missing       IR1   

  Land Contour Utilities Lot Config  ... Pool Area  Pool QC    Fence  \
0          Lvl    AllPub     Corner  ...         0  Missing  Missing   
1          Lvl    AllPub     Inside  ...         0  Missing    MnPrv   
2          Lvl    AllPub     Corner  ...         0  Missing  Missing   
3          Lvl    AllPub     Corner  

In [19]:
# ============================================================================
# CELL 7: Save Data Schema
# ============================================================================
print("\n" + "="*70)
print("SAVING DATA SCHEMA")
print("="*70)

data_path = Path('/Users/karthika/housing_app_fall25/data')
data_path.mkdir(exist_ok=True, parents=True)

# Get feature lists
numeric_features = [col for col in df_clean.select_dtypes(include=[np.number]).columns if col != 'PriceCategory']
categorical_features = df_clean.select_dtypes(include=['object']).columns.tolist()

# Calculate price ranges for each category
price_ranges = []
for cat in range(4):
    cat_prices = df[df['PriceCategory'] == cat]['SalePrice']
    price_ranges.append({
        'category': cat,
        'min': float(cat_prices.min()),
        'max': float(cat_prices.max()),
        'mean': float(cat_prices.mean()),
        'count': int(len(cat_prices))
    })

# Create schema
schema = {
    "target": "PriceCategory",
    "target_mapping": {
        "0": f"Low (${price_ranges[0]['min']:,.0f} - ${price_ranges[0]['max']:,.0f})",
        "1": f"Medium (${price_ranges[1]['min']:,.0f} - ${price_ranges[1]['max']:,.0f})",
        "2": f"High (${price_ranges[2]['min']:,.0f} - ${price_ranges[2]['max']:,.0f})",
        "3": f"Very High (${price_ranges[3]['min']:,.0f} - ${price_ranges[3]['max']:,.0f})"
    },
    "price_ranges": price_ranges,
    "numeric_features": numeric_features,
    "categorical_features": categorical_features,
    "total_features": len(df_clean.columns) - 1,
    "total_samples": len(df_clean),
    "database_path": "db/housing_classification.db",
    "table_name": "housing_data",
    "quartiles": {
        "q1": float(q1),
        "q2": float(q2),
        "q3": float(q3)
    }
}

# Save schema
schema_path = data_path / 'data_schema.json'
with open(schema_path, 'w') as f:
    json.dump(schema, f, indent=2)

print(f"✅ Schema saved: {schema_path}")
print("\n📊 Schema Summary:")
print(f"   Total Features: {schema['total_features']}")
print(f"   Numeric Features: {len(numeric_features)}")
print(f"   Categorical Features: {len(categorical_features)}")
print(f"   Target Classes: 4")
print("\n💰 Price Categories:")
for key, value in schema['target_mapping'].items():
    print(f"   Class {key}: {value}")

print("\n" + "="*70)
print("✅ DATABASE SETUP COMPLETE!")
print("="*70)



SAVING DATA SCHEMA
✅ Schema saved: /Users/karthika/housing_app_fall25/data/data_schema.json

📊 Schema Summary:
   Total Features: 79
   Numeric Features: 36
   Categorical Features: 43
   Target Classes: 4

💰 Price Categories:
   Class 0: Low ($12,789 - $129,500)
   Class 1: Medium ($129,800 - $160,000)
   Class 2: High ($160,200 - $213,500)
   Class 3: Very High ($213,750 - $755,000)

✅ DATABASE SETUP COMPLETE!
